In [1]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas

Thu Aug 22 2019 

CPython 3.6.8
IPython 7.6.0

numpy 1.16.4
scipy 1.2.1
sklearn 0.21.2
pandas 0.24.2

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-58-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [2]:
%matplotlib inline
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import os
import nolds
import data
import mne
from random import randint

from config import *
from data.utils import prepare_dfs, prepare_resp_non, prepare_dep_non, get_metapkl
from data.data_files import files_builder, DataKind
from classification.prediction import predict
from classification.scorers import scorer_factory
    
metapkl = get_metapkl()
meta_df = pd.read_excel(os.path.join(RAW_ROOT, META_FILE_NAME), index_col='ID', names=META_COLUMN_NAMES)
data = np.transpose(files_builder(DataKind('processed')).single_file('1a.fif').df.values)

Opening raw data file /home/miroslav/Source/thesis_project/data/processed/1a.fif...
This filename (/home/miroslav/Source/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
Isotrak not found
    Range : 0 ... 19104 =      0.000 ...    76.416 secs
Ready.


/home/miroslav/Source/thesis_project/src/data/utils.py:18: RuntimeWarning: This filename (/home/miroslav/Source/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  raw_fif = mne.io.read_raw_fif(file_path)


In [3]:
def f(row, col, t1, t2=0):
    if row[col] <= t1:
        return -1
    if row[col] <= t2:
        return 0
    else:
        return 1
    
metapkl['dep'] = metapkl.apply(lambda row: f(row, 'sc', 16, 28), axis=1)
metapkl['dep_bef'] = metapkl.apply(lambda row: f(row, 'sc_bef', 27), axis=1)
metapkl['dep_aft'] = metapkl.apply(lambda row: f(row, 'sc_aft', 16, 28), axis=1)
metapkl = metapkl.astype({'dep': 'category', 'dep_bef': 'category', 'dep_aft': 'category'})                                           
print(metapkl['dep'].value_counts())
print(metapkl['dep_bef'].value_counts())
print(metapkl['dep_aft'].value_counts())

 0    148
-1     62
 1     56
Name: dep, dtype: int64
-1    142
 1    124
Name: dep_bef, dtype: int64
 0    126
-1    124
 1     16
Name: dep_aft, dtype: int64


In [4]:
from itertools import combinations
from sklearn import svm, datasets, metrics
from sklearn.feature_selection import (SelectFromModel, RFE, RFECV, SelectKBest, mutual_info_classif, chi2)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from genetic_selection import GeneticSelectionCV
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

estimators = {
    svm.SVC(kernel='linear', class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 1.5, 2, 3, 4, 5, 10, 50],
        'kernel': ('linear', 'poly', 'rbf'),
        'gamma': ('auto', 'scale'),
        # 'decision_function_shape' : ('ovo', 'ovr'),
    },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 2, 3, 4, 5, 10, 20, 50],
        'penalty': ['l2', 'l1'],
    },
}

def get_selectors(estimator):
    return {
        'RFECV': RFECV(estimator, 5), 
        'SelectFromModel': SelectFromModel(estimator), 
        # 'SelectKBest': SelectKBest(chi2, 3),
        'Genetic': GeneticSelectionCV(estimator,
                                      cv=5,
                                      verbose=0,
                                      scoring=scorer,
                                      n_population=80,
                                      crossover_proba=0.8,
                                      mutation_proba=0.2,
                                      n_generations=80,
                                      crossover_independent_proba=0.5,
                                      mutation_independent_proba=0.05,
                                      tournament_size=5,
                                      caching=True,
                                      n_jobs=1
                                     ),
    }

scorer = scorer_factory(metrics.roc_auc_score, average='weighted')
features = ('corr', 'lyap', 'sampen', 'dfa', 'hurst', 'higu')
comb_size = 1
grid_search_cv = 5

for estimator, params in estimators.items():
    for cols in combinations(features, comb_size):
        for selector_name, selector in get_selectors(estimator).items():
            print(cols)
            print(selector_name)
            gs = GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=grid_search_cv)
            predict('dep_bef', 'a', cols, estimator, metapkl, gs=gs, show_selected=True, selector=selector)
            print(gs.best_params_)
            print('\n\n')

/home/miroslav/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/home/miroslav/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


('corr',)
RFECV
[('FP1', 'corr'), ('FP2', 'corr'), ('F3', 'corr'), ('F4', 'corr'), ('C3', 'corr'), ('C4', 'corr'), ('P3', 'corr'), ('P4', 'corr'), ('O1', 'corr'), ('O2', 'corr'), ('F7', 'corr'), ('F8', 'corr'), ('T3', 'corr'), ('T4', 'corr'), ('T5', 'corr'), ('T6', 'corr'), ('Fz', 'corr'), ('Cz', 'corr'), ('Pz', 'corr')]
Class distribution:  {-1: 65, 1: 55}
Accuracy: 0.62 (+/- 0.07)
Precision: 0.62 (+/- 0.08)
Recall: 0.62 (+/- 0.07)
F1: 0.61 (+/- 0.07)
0.62 $\pm$ 0.07 & 0.62 $\pm$ 0.08 & 0.62 $\pm$ 0.07 & 0.61 $\pm$ 0.07 &  \\ \hline
{'C': 1.0, 'gamma': 'auto', 'kernel': 'linear'}



('corr',)
SelectFromModel
[('FP1', 'corr'), ('F3', 'corr'), ('F4', 'corr'), ('C4', 'corr'), ('T3', 'corr'), ('T4', 'corr'), ('T5', 'corr'), ('Fz', 'corr'), ('Pz', 'corr')]
Class distribution:  {-1: 65, 1: 55}
Accuracy: 0.66 (+/- 0.07)
Precision: 0.67 (+/- 0.08)
Recall: 0.66 (+/- 0.07)
F1: 0.63 (+/- 0.09)
0.66 $\pm$ 0.07 & 0.67 $\pm$ 0.08 & 0.66 $\pm$ 0.07 & 0.63 $\pm$ 0.09 &  \\ \hline
{'C': 10, 'gamma': '

/home/miroslav/anaconda3/envs/thesis/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[('FP2', 'lyap')]
Class distribution:  {-1: 65, 1: 55}
Accuracy: 0.58 (+/- 0.10)
Precision: 0.59 (+/- 0.11)
Recall: 0.58 (+/- 0.10)
F1: 0.58 (+/- 0.11)
0.58 $\pm$ 0.10 & 0.59 $\pm$ 0.11 & 0.58 $\pm$ 0.10 & 0.58 $\pm$ 0.11 &  \\ \hline
{'C': 50, 'gamma': 'scale', 'kernel': 'rbf'}



('lyap',)
SelectFromModel
[('FP2', 'lyap'), ('F4', 'lyap'), ('C4', 'lyap'), ('P4', 'lyap'), ('O1', 'lyap'), ('F7', 'lyap'), ('F8', 'lyap'), ('T4', 'lyap'), ('T6', 'lyap')]


KeyboardInterrupt: 

# CROSS-VALIDATED

In [10]:
estimators = [
    # LogisticRegression(class_weight='balanced'),
    svm.SVC(C=1, class_weight='balanced', kernel='linear'),
]
# 74
channels = [('C4', 'lyap'), ('T3', 'lyap'), ('T6', 'lyap'), ('Pz', 'lyap'), ('C3', 'dfa'), ('P4', 'dfa')]
channels = [('F3', 'lyap'), ('F7', 'lyap'), ('T6', 'lyap'), ('O1', 'corr'), ('O2', 'corr'), ('T5', 'corr')]
# 73
# channels = [('P3', 'lyap'), ('T3', 'lyap'), ('T6', 'lyap'), ('Pz', 'lyap'), ('C3', 'hurst'), ('T3', 'hurst')] # C=50
# channels = [('T6', 'lyap'), ('Fz', 'lyap'), ('O2', 'sampen'), ('T6', 'sampen'), ('P4', 'dfa')]
# channels = [('F3', 'corr'), ('Fz', 'corr'), ('P3', 'higu'), ('Cz', 'higu')]

for estimator in estimators:
    est = predict('dep', None, None, estimator, channels=channels)

('dep', None, None, SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)) {'channels': [('F3', 'lyap'), ('F7', 'lyap'), ('T6', 'lyap'), ('O1', 'corr'), ('O2', 'corr'), ('T5', 'corr')]}
Class distribution:  {-1: 84, 1: 78}
Accuracy: 0.55 (+/- 0.09)
Precision: 0.55 (+/- 0.10)
Recall: 0.55 (+/- 0.09)
F1: 0.54 (+/- 0.09)
0.55 $\pm$ 0.09 & 0.55 $\pm$ 0.10 & 0.55 $\pm$ 0.09 & 0.54 $\pm$ 0.09 &  \\ \hline


In [6]:
estimators = [
    # LogisticRegression(class_weight='balanced'),
    svm.SVC(C=0.5, class_weight='balanced', kernel='linear'),
]
# 72
channels = [('T3', 'lyap'), ('T5', 'lyap'), ('T6', 'lyap'), ('Pz', 'lyap')]

for estimator in estimators:
    est = predict('dep', None, None, estimator, channels=channels)

Class distribution:  {-1: 49, 1: 48}
Accuracy: 0.72 (+/- 0.04)
Precision: 0.73 (+/- 0.04)
Recall: 0.72 (+/- 0.04)
F1: 0.72 (+/- 0.04)
0.72 $\pm$ 0.04 & 0.73 $\pm$ 0.04 & 0.72 $\pm$ 0.04 & 0.72 $\pm$ 0.04 &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [32]:
estimators = [
    # LogisticRegression(class_weight='balanced'),
    svm.SVC(C=1, class_weight='balanced', kernel='linear'),
]
# 71
channels = [('F3', 'corr'), ('C4', 'corr'), ('P3', 'corr'), 
            ('F8', 'corr'), ('T5', 'corr'), ('T6', 'corr'), ('Fz', 'corr'), ('Cz', 'corr')]

for estimator in estimators:
    est = predict('dep', None, None, estimator, show_selected=True, channels=channels)

Class distribution:  {-1: 49, 1: 48}
Accuracy: 0.71 (+/- 0.05)
Precision: 0.72 (+/- 0.05)
Recall: 0.71 (+/- 0.05)
F1: 0.71 (+/- 0.05)


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [32]:
estimators = [
    LogisticRegression(C=2, penalty='l2', class_weight='balanced'),
    # svm.SVC(C=1, class_weight='balanced', kernel='linear'),
]

# 67
channels = [('F8', 'dfa'), ('O2', 'dfa'),]

for estimator in estimators:
    est = predict('dep', None, None, estimator, channels=channels)

Class distribution:  {-1: 49, 1: 48}
Accuracy: 0.67 (+/- 0.16)
Precision: 0.68 (+/- 0.17)
Recall: 0.67 (+/- 0.16)
F1: 0.67 (+/- 0.16)
0.67 $\pm$ 0.16 & 0.68 $\pm$ 0.17 & 0.67 $\pm$ 0.16 & 0.67 $\pm$ 0.16 &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [31]:
estimators = [
    # LogisticRegression(C=2, penalty='l2', class_weight='balanced'),
    svm.SVC(C=3, class_weight='balanced', kernel='rbf'),
]

# 67
channels = [('C3', 'higu'), ('C4', 'higu'), ('P4', 'higu'), ('T6', 'higu'), ('Cz', 'higu')]

for estimator in estimators:
    est = predict('dep', None, None, estimator, channels=channels)

Class distribution:  {-1: 49, 1: 48}
Accuracy: 0.67 (+/- 0.11)
Precision: 0.67 (+/- 0.12)
Recall: 0.67 (+/- 0.11)
F1: 0.67 (+/- 0.11)
0.67 $\pm$ 0.11 & 0.67 $\pm$ 0.12 & 0.67 $\pm$ 0.11 & 0.67 $\pm$ 0.11 &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [30]:
estimators = [
    LogisticRegression(C=1, penalty='l2', class_weight='balanced'),
    # svm.SVC(C=1, class_weight='balanced', kernel='linear'),
]

# 68
channels = [('C4', 'sampen'), ('O2', 'sampen'), ('T6', 'sampen'),]

for estimator in estimators:
    est = predict('dep', None, None, estimator, channels=channels)

Class distribution:  {-1: 49, 1: 48}
Accuracy: 0.68 (+/- 0.12)
Precision: 0.69 (+/- 0.12)
Recall: 0.68 (+/- 0.12)
F1: 0.68 (+/- 0.12)
0.68 $\pm$ 0.12 & 0.69 $\pm$ 0.12 & 0.68 $\pm$ 0.12 & 0.68 $\pm$ 0.12 &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [33]:
estimators = [
    LogisticRegression(C=4, penalty='l2', class_weight='balanced'),
    # svm.SVC(C=1, class_weight='balanced', kernel='linear'),
]

# 67
channels = [('O2', 'hurst'), ('T4', 'hurst'),]

for estimator in estimators:
    est = predict('dep', None, None, estimator, channels=channels)

Class distribution:  {-1: 49, 1: 48}
Accuracy: 0.67 (+/- 0.17)
Precision: 0.68 (+/- 0.18)
Recall: 0.67 (+/- 0.17)
F1: 0.67 (+/- 0.17)
0.67 $\pm$ 0.17 & 0.68 $\pm$ 0.18 & 0.67 $\pm$ 0.17 & 0.67 $\pm$ 0.17 &  \\ \hline


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [29]:
channels = [('T3', 'lyap'), ('F4', 'lyap'), ('T6', 'lyap'), ('P3', 'lyap'), ('O2', 'lyap'), ('Pz', 'lyap')]
channels = [('F3', 'higu'), ('C3', 'higu'), ('C3', 'higu'), ('C4', 'higu'), ('P4', 'higu'), ('T6', 'higu'), ('Cz', 'higu'),]
channels = [('C4', 'sampen'), ('O2', 'sampen'), ('T6', 'sampen'),]
channels = [('F8', 'hurst'), ('O2', 'hurst'), ('T6', 'hurst'), ('T4', 'hurst'),]

channels = [('P3', 'lyap'), ('T3', 'lyap'), ('T6', 'lyap'), ('Pz', 'lyap'), ('C3', 'hurst'), ('T3', 'hurst')]
channels = [('T6', 'lyap'), ('Fz', 'lyap'), ('O2', 'sampen'), ('T6', 'sampen'), ('P4', 'dfa')]
channels = [('F3', 'corr'), ('Fz', 'corr'), ('P3', 'higu'), ('Cz', 'higu')]
channels = [('F3', 'corr'), ('C4', 'corr'), ('P3', 'corr'), 
            ('F8', 'corr'), ('T5', 'corr'), ('T6', 'corr'), ('Fz', 'corr'), ('Cz', 'corr')]

estimators = {
    svm.SVC(kernel='linear', class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 1.5, 2, 3, 4, 5, 10, 50],
        'kernel': ('linear', 'poly', 'rbf'),
        # 'decision_function_shape' : ('ovo', 'ovr'),
    },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 2, 3, 4, 5, 10, 20, 50],
        'penalty': ['l2', 'l1'],
    },
}

for estimator, params in estimators.items():
    gs = grid_search.GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=5)
    selector = GeneticSelectionCV(estimator,
                                      cv=5,
                                      verbose=0,
                                      scoring=scorer,
                                      n_population=80,
                                      crossover_proba=0.8,
                                      mutation_proba=0.2,
                                      n_generations=80,
                                      crossover_independent_proba=0.5,
                                      mutation_independent_proba=0.05,
                                      tournament_size=5,
                                      caching=True,
                                      n_jobs=-1)
    est = predict('dep', None, None, estimator, show_selected=True, channels=channels, gs=gs, selector=selector)
    print(gs.best_params_)

/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


[('F3', 'corr'), ('C4', 'corr'), ('P3', 'corr'), ('F8', 'corr'), ('T5', 'corr'), ('T6', 'corr'), ('Fz', 'corr'), ('Cz', 'corr')]
Class distribution:  {-1: 49, 1: 48}
Accuracy: 0.71 (+/- 0.05)
Precision: 0.72 (+/- 0.05)
Recall: 0.71 (+/- 0.05)
F1: 0.71 (+/- 0.05)
0.71 $\pm$ 0.05 & 0.72 $\pm$ 0.05 & 0.71 $\pm$ 0.05 & 0.71 $\pm$ 0.05 &  \\ \hline
{'C': 1.0, 'kernel': 'linear'}


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


[('F3', 'corr'), ('C4', 'corr'), ('F8', 'corr'), ('T5', 'corr'), ('Fz', 'corr'), ('Cz', 'corr')]
Class distribution:  {-1: 49, 1: 48}
Accuracy: 0.69 (+/- 0.07)
Precision: 0.70 (+/- 0.08)
Recall: 0.69 (+/- 0.07)
F1: 0.69 (+/- 0.07)
0.69 $\pm$ 0.07 & 0.70 $\pm$ 0.08 & 0.69 $\pm$ 0.07 & 0.69 $\pm$ 0.07 &  \\ \hline
{'C': 3, 'penalty': 'l2'}


# Healthy below 10, depressed above 30 (26 / 30)

## Hurst exponent

In [69]:
estimator = LogisticRegression(C=3.5, penalty='l2', class_weight='balanced')
predict('dep', None, ('hurst',), estimator, channels=('O1', 'T4'))

Training distribution:  {-1: 20, 1: 19}
Testing distribution:  {-1: 6, 1: 11}
Accuracy score: 0.53
Confusion matrix:
 [[5 1]
 [7 4]]
Precision score:  0.6647058823529413
Recall score:  0.5294117647058824
f1 score:  0.5196078431372549
ROC AUC score:  0.5984848484848485
Accuracy: 0.65 (+/- 0.28)


LogisticRegression(C=3.5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## DFA

In [70]:
estimator = LogisticRegression(C=3.5, penalty='l2', class_weight='balanced')
# predict('dep', None, ('dfa',), estimator, channels=('O1', 'T4')) # DFA_4_300_11
# predict('dep', None, ('dfa',), estimator, channels=('F3', 'C3', 'C4', 'O1', 'T4', 'Pz')) # DFA_4_300_11_offset
predict('dep', None, ('dfa',), estimator, channels=('C3', 'C4', 'O1', 'T4')) # DFA_4_300_11_offset

Training distribution:  {-1: 20, 1: 19}
Testing distribution:  {-1: 6, 1: 11}
Accuracy score: 0.53
Confusion matrix:
 [[5 1]
 [7 4]]
Precision score:  0.6647058823529413
Recall score:  0.5294117647058824
f1 score:  0.5196078431372549
ROC AUC score:  0.5984848484848485
Accuracy: 0.72 (+/- 0.34)


LogisticRegression(C=3.5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## Correlation dimension

In [71]:
estimator = svm.SVC(C=0.02, kernel='poly', decision_function_shape='ovo', class_weight='balanced', shrinking=True)
predict('dep', None, ('corr',), estimator, channels=('F3', 'F4', 'P3', 'O1', 'T4', 'T5', 'Cz', 'Pz'))

Training distribution:  {-1: 20, 1: 19}
Testing distribution:  {-1: 6, 1: 11}
Accuracy score: 0.35
Confusion matrix:
 [[4 2]
 [9 2]]
Precision score:  0.4321266968325792
Recall score:  0.35294117647058826
f1 score:  0.3211558307533539
ROC AUC score:  0.4242424242424243
Accuracy: 0.62 (+/- 0.22)


SVC(C=0.02, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## Lyapunov exponent
I was unable to create a classifier with better than 65% acc., although the depression scores and LLE are clearly correlated.

In [72]:
estimator = svm.SVC(C=50, kernel='linear', decision_function_shape='ovo', class_weight='balanced')
predict('dep', None, ('lyap',), estimator, channels=('FP1', 'FP2', 'F4', 'C3', 'O1', 'T3', 'T6'))

Training distribution:  {-1: 20, 1: 19}
Testing distribution:  {-1: 6, 1: 11}
Accuracy score: 0.53
Confusion matrix:
 [[2 4]
 [4 7]]
Precision score:  0.5294117647058824
Recall score:  0.5294117647058824
f1 score:  0.5294117647058824
ROC AUC score:  0.48484848484848486
Accuracy: 0.64 (+/- 0.22)


SVC(C=50, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [73]:
estimator = svm.SVC(C=1, kernel='rbf', decision_function_shape='ovo', class_weight='balanced')
predict('dep', None, ('lyap',), estimator, channels=('T6',))

Training distribution:  {-1: 20, 1: 19}
Testing distribution:  {-1: 6, 1: 11}
Accuracy score: 0.47
Confusion matrix:
 [[3 3]
 [6 5]]
Precision score:  0.5220588235294118
Recall score:  0.47058823529411764
f1 score:  0.48173374613003095
ROC AUC score:  0.4772727272727273
Accuracy: 0.66 (+/- 0.24)


SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## Sample entropy

In [74]:
estimator = LogisticRegression(C=2, penalty='l2', class_weight='balanced')
predict('dep', None, ('sampen',), estimator, channels=('P3', 'P4', 'T5', 'T6'))

Training distribution:  {-1: 20, 1: 19}
Testing distribution:  {-1: 6, 1: 11}
Accuracy score: 0.53
Confusion matrix:
 [[4 2]
 [6 5]]
Precision score:  0.6033613445378152
Recall score:  0.5294117647058824
f1 score:  0.5359477124183006
ROC AUC score:  0.5606060606060607
Accuracy: 0.66 (+/- 0.16)


LogisticRegression(C=2, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## Higuchi dimension
Bad, although, again, correlated

# Healthy below 12, depressed above 28 (56 / 51)
Correlation, DFA, Hurst preserve their performance to some extent. Hurst only 67%.

## Correlation dimension

In [7]:
estimator = LogisticRegression(C=0.5, penalty='l2', class_weight='balanced')
predict('dep', None, ('corr',), estimator, channels=('F3', 'F4', 'T3', 'T5', 'Fz', 'Cz'))

Training distribution:  {-1: 31, 1: 29}
Testing distribution:  {-1: 8, 1: 19}
Accuracy score: 0.70
Confusion matrix:
 [[ 7  1]
 [ 7 12]]
Precision score:  0.7977207977207977
Recall score:  0.7037037037037037
f1 score:  0.7163299663299663
ROC AUC score:  0.7532894736842105
Accuracy: 0.63 (+/- 0.12)


LogisticRegression(C=0.5, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## DFA

In [8]:
estimator = svm.SVC(C=1.5, kernel='rbf', decision_function_shape='ovo', class_weight='balanced')
predict('dep', None, ('dfa',), estimator, channels=('O1', 'T5', 'T6'))

Training distribution:  {-1: 31, 1: 29}
Testing distribution:  {-1: 8, 1: 19}
Accuracy score: 0.52
Confusion matrix:
 [[ 6  2]
 [11  8]]
Precision score:  0.6675381263616558
Recall score:  0.5185185185185185
f1 score:  0.5304725415070243
ROC AUC score:  0.5855263157894737
Accuracy: 0.44 (+/- 0.16)


SVC(C=1.5, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## Hurst exponent

In [9]:
estimator = svm.SVC(C=1.5, kernel='rbf', decision_function_shape='ovo', class_weight='balanced', shrinking=True)
predict('dep', None, ('hurst',), estimator, channels=('C4', 'P4', 'T4', 'T6', 'Cz'))

Training distribution:  {-1: 31, 1: 29}
Testing distribution:  {-1: 8, 1: 19}
Accuracy score: 0.67
Confusion matrix:
 [[ 3  5]
 [ 4 15]]
Precision score:  0.6547619047619047
Recall score:  0.6666666666666666
f1 score:  0.6598290598290597
ROC AUC score:  0.5822368421052632
Accuracy: 0.43 (+/- 0.39)


SVC(C=1.5, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Manual selection

## LLE

## Higuchi fractal dimension

In [44]:
estimators = {
    # svm.SVC(kernel='linear', class_weight='balanced'): {
    #     'C': [0.02, 0.5, 1.0, 1.5, 2, 3, 4, 5, 10, 50],
    #     'kernel': ('linear', 'poly', 'rbf'),
    #     # 'decision_function_shape' : ('ovo', 'ovr'),
    # },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 2, 3, 4, 5, 10, 20, 50],
        'penalty': ['l2', 'l1'],
    },
}
channels = ['T6']
for estimator, params in estimators.items():
    est = grid_search.GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=4)
    est = predict('dep', None, ('higu',), est, show_selected=True, channels=channels, seed=213, eval_cv=False)

/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


Training distribution:  {-1: 51, 1: 48}
Testing distribution:  {-1: 22, 1: 21}
Accuracy score: 0.56
Confusion matrix:
 [[11 11]
 [ 8 13]]
Precision score:  0.5607405140758874
Recall score:  0.5581395348837209
f1 score:  0.5567025902817168
ROC AUC score:  0.5595238095238095


# # DFA

In [63]:
estimators = {
    # svm.SVC(kernel='linear', class_weight='balanced'): {
    #     'C': [0.02, 0.5, 1.0, 1.5, 2, 3, 4, 5, 10, 50],
    #     'kernel': ('linear', 'poly', 'rbf'),
    #     'decision_function_shape' : ('ovo', 'ovr'),
    # },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 2, 3, 4, 5, 10, 20, 50],
        'penalty': ['l2', 'l1'],
    },
}
seed = randint(0, 100000)
print('Seed: %s' % seed)
channels = ['T4']
for estimator, params in estimators.items():
    est = grid_search.GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=4)
    est = predict('dep', None, ('dfa',), est, show_selected=True, channels=channels, seed=seed)

Seed: 14411
Training distribution:  {-1: 36, 1: 31}
Testing distribution:  {-1: 13, 1: 17}
Accuracy score: 0.70
Confusion matrix:
 [[ 8  5]
 [ 4 13]]
Precision score:  0.6981481481481481
Recall score:  0.7
f1 score:  0.6982857142857142
ROC AUC score:  0.6900452488687784


## Sample entropy

In [65]:
estimators = {
    # svm.SVC(kernel='linear', class_weight='balanced'): {
    #     'C': [0.02, 0.5, 1.0, 1.5, 2, 3, 4, 5, 10, 50],
    #     'kernel': ('linear', 'poly', 'rbf'),
    #     'decision_function_shape' : ('ovo', 'ovr'),
    # },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 2, 3, 4, 5, 10, 20, 50],
        'penalty': ['l2', 'l1'],
    },
}
seed = randint(0, 100000)
print('Seed: %s' % seed)
channels = ['T6']
for estimator, params in estimators.items():
    est = grid_search.GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=4)
    est = predict('dep', None, ('sampen',), est, show_selected=True, channels=channels, seed=seed)

Seed: 67493
Training distribution:  {-1: 33, 1: 34}
Testing distribution:  {-1: 16, 1: 14}
Accuracy score: 0.77
Confusion matrix:
 [[12  4]
 [ 3 11]]
Precision score:  0.7688888888888888
Recall score:  0.7666666666666667
f1 score:  0.7669262143121988
ROC AUC score:  0.7678571428571428


## Hurst exponent

In [69]:
estimators = {
    # svm.SVC(kernel='linear', class_weight='balanced'): {
    #     'C': [0.02, 0.5, 1.0, 1.5, 2, 3, 4, 5, 10, 50],
    #     'kernel': ('linear', 'poly', 'rbf'),
    #     'decision_function_shape' : ('ovo', 'ovr'),
    # },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 2, 3, 4, 5, 10, 20, 50],
        'penalty': ['l2', 'l1'],
    },
}
seed = randint(0, 100000)
print('Seed: %s' % seed)
channels = ['T4']
for estimator, params in estimators.items():
    est = grid_search.GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=4)
    est = predict('dep', None, ('hurst',), est, show_selected=True, channels=channels, seed=seed)

Seed: 32420
Training distribution:  {-1: 36, 1: 31}
Testing distribution:  {-1: 13, 1: 17}
Accuracy score: 0.70
Confusion matrix:
 [[ 8  5]
 [ 4 13]]
Precision score:  0.6981481481481481
Recall score:  0.7
f1 score:  0.6982857142857142
ROC AUC score:  0.6900452488687784


## Correlation dimension

In [34]:
estimators = {
    # svm.SVC(kernel='linear', class_weight='balanced'): {
    #     'C': [0.02, 0.5, 1.0, 1.5, 2, 3, 4, 5, 10, 50],
    #     'kernel': ('linear', 'poly', 'rbf'),
    #     'decision_function_shape' : ('ovo', 'ovr'),
    # },
    LogisticRegression(class_weight='balanced'): {
        'C': [0.02, 0.5, 1.0, 2, 3, 4, 5, 10, 20, 50],
        'penalty': ['l2', 'l1'],
    },
}
# 36622
channels = ['C3']
for estimator, params in estimators.items():
    # est = grid_search.GridSearchCV(estimator, params, iid=False, scoring=scorer, cv=4)
    est = predict('dep', None, ('corr',), estimator, show_selected=True, channels=channels)

Seed: 5648
Accuracy: 0.58 (+/- 0.14)
Precision: 0.59 (+/- 0.15)
Recall: 0.58 (+/- 0.14)
F1: 0.58 (+/- 0.15)


/home/kovar/miniconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)
